<a href="https://colab.research.google.com/github/vmagapu/genai-projects/blob/main/Personal_Fiance_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 1. INSTALL THE MISSING LIBRARY (Must be the first line)
!pip install xlsxwriter

import pandas as pd
import numpy as np

# 2. Load Data
try:
    df_trans = pd.read_csv('Networth-Cashflow.xlsx - All Transactions.csv')
    df_inv = pd.read_csv('Networth-Cashflow.xlsx - Investment Positions.csv', header=1)
except Exception as e:
    print("Error: Please upload the CSV files to the Files sidebar first!")
    raise e

# --- CLEANING FUNCTIONS ---
def clean_currency(x):
    """Removes $ , ) ( and converts to float"""
    if isinstance(x, str):
        clean_str = x.replace('$', '').replace(',', '').strip()
        if clean_str.startswith('(') and clean_str.endswith(')'):
            clean_str = '-' + clean_str[1:-1]
        return clean_str
    return x

# Apply cleaning
df_trans['Amount'] = df_trans['Amount'].apply(clean_currency)
df_trans['Amount'] = pd.to_numeric(df_trans['Amount'], errors='coerce').fillna(0)

cols_to_clean = ['Shares', 'Price', 'Cost', 'Values']
for col in cols_to_clean:
    if col in df_inv.columns:
        df_inv[col] = df_inv[col].apply(clean_currency)
        df_inv[col] = pd.to_numeric(df_inv[col], errors='coerce').fillna(0)

# --- PROCESS TRANSACTIONS ---
# 'dayfirst=False' helps with mixed formats, but 'ISO' is best if available.
# We use errors='coerce' to skip bad rows without crashing.
df_trans['Date'] = pd.to_datetime(df_trans['Date'], errors='coerce')
df_trans = df_trans.sort_values(by='Date')

df_trans['Debit Amount'] = df_trans['Amount'].apply(lambda x: abs(x) if x < 0 else 0)
df_trans['Credit Amount'] = df_trans['Amount'].apply(lambda x: x if x > 0 else 0)
df_trans['Balance'] = df_trans['Amount'].cumsum()

# Target DataFrame for Transactions
df_trans_final = pd.DataFrame()
df_trans_final['Date'] = df_trans['Date']
df_trans_final['Description'] = df_trans['Description']
df_trans_final['Category'] = df_trans['Category']
df_trans_final['Debit Amount'] = df_trans['Debit Amount']
df_trans_final['Credit Amount'] = df_trans['Credit Amount']
df_trans_final['Balance'] = df_trans['Balance']
df_trans_final['Account Name'] = df_trans['Account']

# --- PROCESS INVESTMENTS ---
df_inv_final = pd.DataFrame()
df_inv_final['Investment Type'] = df_inv.get('Classification', df_inv.get('Asset Category', ''))
df_inv_final['Instrument Name'] = df_inv.get('Name', '')
df_inv_final['Account / Platform'] = "Unknown"
df_inv_final['Units'] = df_inv.get('Shares', 0)
df_inv_final['Price per Unit'] = df_inv.get('Price', 0)
df_inv_final['Invested Amount'] = df_inv.get('Cost', 0)
df_inv_final['Current Value'] = df_inv.get('Values', 0)
df_inv_final['Asset Category'] = df_inv.get('Classification', '')
df_inv_final = df_inv_final.dropna(subset=['Instrument Name'])

# --- CREATE EXCEL ---
output_file = 'My_Personal_Finance_Dashboard.xlsx'
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
workbook = writer.book

# Formats
fmt_currency = workbook.add_format({'num_format': '[$₹-4009]#,##0.00'})
fmt_date = workbook.add_format({'num_format': 'yyyy-mm-dd'})
fmt_header = workbook.add_format({'bold': True, 'bg_color': '#D3D3D3', 'border': 1})
fmt_bold = workbook.add_format({'bold': True})

# 1. Controls
ws_controls = workbook.add_worksheet('Controls')
ws_controls.write('B2', 'Latest Data Date', fmt_bold)
ws_controls.write_formula('C2', '=MAX(Transactions_Raw[Date])', fmt_date)
ws_controls.write('B4', 'Select Period', fmt_bold)
ws_controls.write('C4', '12 Months')
ws_controls.data_validation('C4', {'validate': 'list', 'source': ['1 Month', '3 Months', '6 Months', '12 Months', '3 Years', '5 Years', 'All History']})
ws_controls.write('B5', 'Start Date Calculation', fmt_bold)
ws_controls.write_formula('C5', '=IFS(C4="1 Month", EDATE(C2,-1), C4="3 Months", EDATE(C2,-3), C4="6 Months", EDATE(C2,-6), C4="12 Months", EDATE(C2,-12), C4="3 Years", EDATE(C2,-36), C4="5 Years", EDATE(C2,-60), TRUE, DATE(2000,1,1))', fmt_date)

# 2. Transactions
df_trans_final.to_excel(writer, sheet_name='Transactions_Raw', index=False)
ws_trans = writer.sheets['Transactions_Raw']
(max_row, max_col) = df_trans_final.shape
ws_trans.add_table(0, 0, max_row, max_col - 1, {'name': 'Transactions_Raw', 'columns': [{'header': col} for col in df_trans_final.columns]})
ws_trans.set_column('A:A', 12, fmt_date)
ws_trans.set_column('D:F', 15, fmt_currency)

# 3. Investments
df_inv_final.to_excel(writer, sheet_name='Investments_Raw', index=False)
ws_inv = writer.sheets['Investments_Raw']
(max_row_inv, max_col_inv) = df_inv_final.shape
ws_inv.add_table(0, 0, max_row_inv, max_col_inv - 1, {'name': 'Investments_Raw', 'columns': [{'header': col} for col in df_inv_final.columns]})
ws_inv.set_column('F:H', 15, fmt_currency)

# 4. Calc_Engine
ws_calc = workbook.add_worksheet('Calc_Engine')
ws_calc.hide()
ws_calc.write('A1', 'KPIs', fmt_header)
ws_calc.write('A2', 'Total Spend')
ws_calc.write_formula('B2', '=SUMIFS(Transactions_Raw[Debit Amount], Transactions_Raw[Date], ">="&Controls!$C$5)', fmt_currency)
ws_calc.write('A3', 'Total Income')
ws_calc.write_formula('B3', '=SUMIFS(Transactions_Raw[Credit Amount], Transactions_Raw[Date], ">="&Controls!$C$5)', fmt_currency)
ws_calc.write('A4', 'Net Cashflow')
ws_calc.write_formula('B4', '=B3-B2', fmt_currency)
ws_calc.write('A5', 'Savings Rate')
ws_calc.write_formula('B5', '=IFERROR(B4/B3, 0)', workbook.add_format({'num_format': '0.0%'}))
ws_calc.write('D1', 'Category Analysis', fmt_header)
ws_calc.write_formula('D2', '=SORT(UNIQUE(Transactions_Raw[Category]))')
ws_calc.write('E1', 'Spend', fmt_header)
ws_calc.write_formula('E2', '=SUMIFS(Transactions_Raw[Debit Amount], Transactions_Raw[Category], D2#, Transactions_Raw[Date], ">="&Controls!$C$5)')

# 5. Dashboard
ws_dash = workbook.add_worksheet('Dashboard')
ws_dash.write('A1', 'PERSONAL FINANCE DASHBOARD', workbook.add_format({'bold': True, 'font_size': 16}))
ws_dash.write('B3', 'Net Cashflow')
ws_dash.write_formula('B4', '=Calc_Engine!B4', fmt_currency)
ws_dash.write('D3', 'Savings Rate')
ws_dash.write_formula('D4', '=Calc_Engine!B5', workbook.add_format({'num_format': '0.0%', 'font_size': 14, 'bold': True}))
ws_dash.write('F3', 'Total Spend (Period)')
ws_dash.write_formula('F4', '=Calc_Engine!B2', fmt_currency)
chart = workbook.add_chart({'type': 'pie'})
chart.add_series({'name': 'Spend by Category', 'categories': '=Calc_Engine!$D$2:$D$15', 'values': '=Calc_Engine!$E$2:$E$15'})
ws_dash.insert_chart('B7', chart)

# 6. Spend Analysis
ws_spend = workbook.add_worksheet('Spend_Analysis')
ws_spend.write('A1', 'Detailed Spend Analysis', fmt_bold)
ws_spend.write_formula('A3', '=Calc_Engine!D2')

# 7. Net Worth
ws_nw = workbook.add_worksheet('Net_Worth')
ws_nw.write('A1', 'Net Worth Snapshot', fmt_bold)
ws_nw.write('A3', 'Investments Value')
ws_nw.write_formula('B3', '=SUM(Investments_Raw[Current Value])', fmt_currency)
ws_nw.write('A4', 'Liquid Assets (Est)')
ws_nw.write_formula('B4', '=SUM(Transactions_Raw[Balance])', fmt_currency)
ws_nw.write('A5', 'Total Net Worth')
ws_nw.write_formula('B5', '=B3+B4', fmt_currency)

# 8. Budgeting
ws_budget = workbook.add_worksheet('Budgeting')
ws_budget.write('A1', 'Budget Planner', fmt_bold)
ws_budget.write('A3', 'Category')
ws_budget.write('B3', 'Avg Spend (Last 12M)')
ws_budget.write('C3', 'Budget (Manual)')
ws_budget.write('D3', 'Variance')
ws_budget.write_formula('A4', '=SORT(UNIQUE(Transactions_Raw[Category]))')
ws_budget.write_formula('B4', '=AVERAGEIFS(Transactions_Raw[Debit Amount], Transactions_Raw[Category], A4#, Transactions_Raw[Date], ">="&EDATE(TODAY(),-12))')

writer.close()
print(f"Success! Refresh the Files sidebar and download {output_file}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.9 MB/s eta 0:00:00


/tmp/ipython-input-1682566072.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_trans['Date'] = pd.to_datetime(df_trans['Date'], errors='coerce')


Success! Refresh the Files sidebar and download My_Personal_Finance_Dashboard.xlsx
